In [1]:
import pandas as pd 
import numpy as np  
import re 
import unicodedata

# csv name
csv = 'input.csv'
# reads the csv
data = pd.read_csv(csv , sep = ';')

# Select columns
data = data[[ 'job_title' , 'job_type' ,'location' , 'salary' ]]


# unicode normalization
def normalize(dataframe):
    columns = data.columns.values
    for x in columns:
        dataframe[x] = dataframe[x].apply(lambda y: unicodedata.normalize('NFD' , str(y)))
        dataframe[x] = dataframe[x].apply(lambda y: unicodedata.normalize('NFC' , str(y)))
        dataframe[x] = dataframe[x].apply(lambda y: unicodedata.normalize('NFKD' , str(y)))
        dataframe[x] = dataframe[x].apply(lambda y: unicodedata.normalize('NFKC', str(y)))
    return dataframe
normalize(data)

,job_title,job_type,location,salary
0,IT Support Technician Job in Madison,Full Time Employee,"Madison, WI 53702",nan
1,Business Reporter/Editor Job in Madison,Full Time,"Madison, WI 53708",nan
2,Johnson & Johnson Family of Companies Job Appl...,"Full Time, Employee",DePuy Synthes Companies is a member of Johnson...,nan
3,Engineer - Quality Job in Dixon,Full Time,"Dixon, CA",nan
4,Shift Supervisor - Part-Time Job in Camphill,Full Time Employee,"Camphill, PA",nan
...,...,...,...,...
21995,Assistant Vice President - Controller Job in C...,Full Time,"Cincinnati, OH","120,000.00 - 160,000.00 $ /yearbonus"
21996,Accountant Job in Cincinnati,Full Time,"Cincinnati, OH 45236","45,000.00 - 60,000.00 $ /year"
21997,AEM/CQ developer Job in Chicago,Full Time,"Chicago, IL 60602",nan
21998,Electrician - Experienced Forging Electrician ...,Full Time Employee,"Chicago, IL 60609",25.00 - 28.00 $ /hour


In [2]:
# Clean job_type columns

# regex pattern. Catches everthing after "Temporary"
pattern = r'\bTemporary.*'
# apply reges
data.loc[: , 'job_type'] = data['job_type'].apply(lambda x: re.sub(pattern, '', str(x), flags=re.IGNORECASE))

# Removes special characters
data.loc[: ,'job_type'] = data['job_type'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', str(x)))

# removes the word "Employee"
data.loc[: ,'job_type'] = data['job_type'].str.replace('Employee', '')
# Removes the specialy character not caught in the regex before
data.loc[: ,'job_type'] = data['job_type'].str.replace('Â', '')
# Removes incorrect "Full Time"
data.loc[: ,'job_type'] = data['job_type'].str.replace('Full TimeÂ', 'Full Time')
# Removes "Type"  
data.loc[: ,'job_type'] = data['job_type'].str.replace('Type', '')
# removes "Job"
data.loc[: ,'job_type'] = data['job_type'].str.replace('Job', '')
# Removes "Exempt"
data.loc[: ,'job_type'] = data['job_type'].str.replace('Exempt', '')
# replaces "Seasonal" with "Temporary"
data.loc[: ,'job_type'] = data['job_type'].str.replace('Seasonal', 'Temporary')

# Removes whitespace
data.loc[: ,'job_type'] = data['job_type'].str.strip()

# Fill empty entries with nan
data.loc[: ,'job_type'] = data['job_type'].fillna(np.nan)

In [3]:
# Clean location column

# regex pattern , should catch only those addresses in the right format
pattern = r"[A-Za-z]+?\s?[A-Za-z]+,\s[A-Za-z][A-Za-z]\s[0-9]{5}"
# Apply the regex pattern to the "location" column
data = data[data['location'].str.contains(r"\b[A-Za-z]+?\s?[A-Za-z]+,\s[A-Za-z][A-Za-z]\s[0-9]{5}\b")]

#data = data[data['location'].str.contains(r"[A-Za-z]+?\s?[A-Za-z]+,\s[A-Za-z][A-Za-z]\s[0-9]{5}")]

# Filter by string lentgh
data = data[data['location'].str.len() <= 30]

# remove 'Address' 
data.loc[: , 'location'] = data['location'].str.replace('Address', '').str.strip()

# Removes whitespace
data.loc[: , 'location'] = data['location'].str.strip()

# fills empty entries with nan
data.replace('', np.nan, inplace=True)

In [4]:
# Clean the job_title column

# the regex is supposed to remove every special characters
data.loc[: , 'job_title'] = data['job_title'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', str(x)))

# clean whitespace
data.loc[: , 'job_title'] = data['job_title'].str.strip()

# regex to delete all string after Job 
pattern = r'\bJob.*'
data.loc[: , 'job_title'] = data['job_title'].apply(lambda x: re.sub(pattern, '', str(x), flags=re.IGNORECASE))

# remove all single characters
data.loc[: , 'job_title'] = data['job_title'].apply(lambda x: re.sub(r'\s\w\s', '', str(x), flags=re.IGNORECASE))

# removes everything after "in" because it is usally city names
pattern = r'\bin.*'
data.loc[: , 'job_title'] = data['job_title'].apply(lambda x: re.sub(pattern, '', str(x), flags=re.IGNORECASE))

# filter by everything that is too small
data = data[data['job_title'].str.len() > 3]

# make all lower case
data.loc[: , 'job_title'] = data['job_title'].apply(lambda x: x.lower())

# Nächster Schritt:
# Die City titel aus "location" column heruasholen und diese strings dann in der "job_title" column ausblenden.
cities = data['location'].apply(lambda x: re.sub(r',\s[A-Za-z][A-Za-z]\s[0-9]{5}', '', str(x))).unique()

# Remove city names from job_title column
for city in cities:
    data.loc[:, 'job_title'] = data['job_title'].apply(lambda x: re.sub(r'\b{}\b'.format(re.escape(city)), '', str(x), flags=re.IGNORECASE))

# remove all single characters
data.loc[: , 'job_title'] = data['job_title'].apply(lambda x: re.sub(r'\s\w\s', '', str(x), flags=re.IGNORECASE))

# Removes any whitespace
data.loc[: , 'job_title'] = data['job_title'].str.strip()

# fills empty entries with nan
data.replace('', np.nan, inplace=True)

In [5]:
# Clean salary column

# remove "401k"
data.loc[: , 'salary'] = data['salary'].str.replace('401K' , '')
data.loc[: , 'salary'] = data['salary'].str.replace('401k' , '')
data.loc[: , 'salary'] = data['salary'].str.replace('401(k)' , '')
data.loc[: , 'salary'] = data['salary'].str.replace('401(K)' , '')

# replace all entries that are exclusivedly characters with nan
# regex pattern
no_digits_pattern = r'^\D*$'
# replace non-digit-entries with nan
data.loc[data['salary'].str.match(no_digits_pattern), 'salary'] = np.nan

# fill blanks with nan
data.loc[: ,'salary'] = data['salary'].fillna(np.nan)

# remove dollar sign
data.loc[: ,'salary'] = data['salary'].str.replace('$', '')
# remove "monthly bonus"
data.loc[: ,'salary'] = data['salary'].str.replace('monthly bonus', '')
data.loc[: ,'salary'] = data['salary'].str.replace('monthlybonus', '')
# remove "up to"
data.loc[: ,'salary'] = data['salary'].str.replace('Up to', '')

# remove all unnecessary characters after the numbers
def remove_chars_after_suffix(salary):
    salary = str(salary)  # Convert to string if not already
    salary = re.sub(r'(?i)/year[^/]*$', '/year', salary)
    salary = re.sub(r'(?i)/month[^/]*$', '/month', salary)
    salary = re.sub(r'(?i)/week[^/]*$', '/week', salary)
    salary = re.sub(r'(?i)/day[^/]*$', '/day', salary)
    salary = re.sub(r'(?i)/hour[^/]*$', '/hour', salary)
    return salary
# apply function
data['salary'] = data['salary'].apply(remove_chars_after_suffix)

# split salary column on "-"
salary_split = data['salary'].str.split('-', n=1 ,expand=True)
# rename new columns
salary_split.columns = ['salary_start', 'salary_end']
# attach new columns to dataframe
data = pd.concat([data, salary_split], axis=1)
# drop the original 'salary' column
data.drop(columns=['salary'], inplace=True)

# define function to get salary type
def get_salary_type(row):
    salary_start = str(row['salary_start'])
    salary_end = str(row['salary_end'])
    
    if 'week' in salary_start or 'week' in salary_end:
        return 'weekly'
    elif 'hour' in salary_start or 'hour' in salary_end:
        return 'hourly'
    elif 'month' in salary_start or 'month' in salary_end:
        return 'monthly'
    elif 'year' in salary_start or 'year' in salary_end:
        return 'yearly'
    else:
        return 'unknown'
# apply function to create new column salary_type
data['salary_type'] = data.apply(get_salary_type, axis=1)

# remove all non-numeric
data['salary_start'] = data['salary_start'].str.extract(r'(\d+)')
data['salary_end'] = data['salary_end'].str.extract(r'(\d+)')
# Convert entries to float
data['salary_start'] = data['salary_start'].astype(float)
data['salary_end'] = data['salary_end'].astype(float)

# allocate value to salary_type where value = 'unknown', based on their amount of value of salary_start or salary_end
def update_salary_type(row):
    if pd.notnull(row['salary_start']):
        if row['salary_start'] > 10000:
            return 'yearly'
        elif 1000 < row['salary_start'] <= 10000:
            return 'monthly'
        elif 100 < row['salary_start'] <= 1000:
            return 'weekly'
        elif row['salary_start'] <= 100:
            return 'hourly'
    elif pd.notnull(row['salary_end']):
        if row['salary_end'] > 10000:
             return 'yearly'
        elif 1000 < row['salary_end'] <= 10000:
            return 'monthly'
        elif 100 < row['salary_end'] <= 1000:
            return 'weekly'
        elif row['salary_end'] <= 100:
            return 'hourly'
    else:
        return row['salary_type']
    return row['salary_type']

# apply function
data['salary_type'] = data.apply(update_salary_type, axis=1)

# create a function that brings all values into the yearly format for easier comparison
def standardize_salary(row):
    # Hourly
    if row['salary_type'] == 'hourly':
        if pd.notnull(row['salary_start']):
            row['salary_start'] *= 1920  # 40 hours/week * 4 weeks * 12 months
            if pd.notnull(row['salary_end']):
                row['salary_end'] *= 1920
        elif pd.notnull(row['salary_end']):
            row['salary_end'] *= 1920  # 40 hours/week * 4 weeks * 12 months
    # Weekly
    elif row['salary_type'] == 'weekly':
        if pd.notnull(row['salary_start']):
            row['salary_start'] *= 48  # 4 weeks * 12 months
            if pd.notnull(row['salary_end']):
                row['salary_end'] *= 48
        elif pd.notnull(row['salary_end']):
            row['salary_end'] *= 48  # 4 weeks * 12 months
    # Monthly
    elif row['salary_type'] == 'monthly':
        if pd.notnull(row['salary_start']):
            row['salary_start'] *= 12  # 12 months
            if pd.notnull(row['salary_end']):
                row['salary_end'] *= 12
        elif pd.notnull(row['salary_end']):
            row['salary_end'] *= 12  # 12 months
    return row
data = data.apply(standardize_salary, axis=1)

# calculate the median of salary_start and salary_end to get a single value
def calculate_median(row):
    salary_start = row['salary_start']
    salary_end = row['salary_end']
    
    # check if both salary_start and salary_end are not null
    if pd.notnull(salary_start) and pd.notnull(salary_end):
        median_salary = (salary_start + salary_end) / 2
        return median_salary
    elif pd.notnull(salary_start):
        return salary_start
    elif pd.notnull(salary_end):
        return salary_end
    else:
        return np.nan
# apply function ot create median_salary column
data.loc[: , 'median_salary'] = data.apply(calculate_median, axis=1)

# Fill in empty values:
# find state initials
def extract_state_initials(location):
    match = re.search(r',\s*([A-Z]{2})\s+\d{5}', location)
    if match:
        return match.group(1)
    else:
        return None
data.loc[: ,'state_initials'] = data['location'].apply(extract_state_initials)

# next calculate the median for each state and fill in the blanks
def calculate_median_salary(data):
    # calculate the median salary for each state
    median_salaries = data.groupby('state_initials')['median_salary'].median()

    # iterate over each state initial
    for state_initials, median_salary in median_salaries.items():
        # Fill in the empty entries in the "median_salary" column with the calculated median salary
        data.loc[data['state_initials'] == state_initials, 'median_salary'] = data.loc[data['state_initials'] == state_initials, 'median_salary'].fillna(median_salary)
    return data
# apply function
data = calculate_median_salary(data)


# drop all unnecessary columns
data.drop(columns=['salary_start'], inplace=True)
data.drop(columns=['salary_type'], inplace=True)
data.drop(columns=['salary_end'], inplace=True)
data.drop(columns=['state_initials'], inplace=True)
# fill al blanks
data.loc[: ,'median_salary'] = data['median_salary'].fillna(np.nan)
# drop any nan 
data.dropna(how='any')

,job_title,job_type,location,median_salary
0,it support technician,Full Time,"Madison, WI 53702",43200.0
1,business reportereditor,Full Time,"Madison, WI 53708",43200.0
7,mailroom clerk,Full Time,"Austin, TX 73301",71040.0
8,housekeeper,Part Time,"Austin, TX 78746",71040.0
10,aflac,Full Time,"Berryville, VA 22611",115200.0
...,...,...,...,...
21993,patient access representative,Full Time,"Chicago, IL 60603",75840.0
21994,immediate customer service position,Part Time,"Cincinnati, OH 45202",73920.0
21996,accountant,Full Time,"Cincinnati, OH 45236",100800.0
21997,aemcq developer,Full Time,"Chicago, IL 60602",75840.0


In [6]:
data.to_csv('output.csv' , sep=';')

In [7]:
print(data.columns)

Index(['job_title', 'job_type', 'location', 'median_salary'], dtype='object')


In [8]:
data.shape

(11072, 4)

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11072 entries, 0 to 21998
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   job_title      11071 non-null  object 
 1   job_type       10823 non-null  object 
 2   location       11072 non-null  object 
 3   median_salary  11039 non-null  float64
dtypes: float64(1), object(3)
memory usage: 432.5+ KB


In [10]:
data.describe()

,median_salary
count,11039.000000
mean,71305.757406
std,34450.069382
min,960.000000
25%,52000.000000
50%,72000.000000
75%,76800.000000
max,921600.000000
